In [1]:
def getTitle(soup,Rank=1):            
    titleDiv = soup.find(id = 'results_objectname'+str(Rank))
    titleText = titleDiv.find('a').text    
    return titleText
    
    


In [2]:
def getImageURL(soup,rank=1):
    BGGID=getBGGID(soup,rank)
    request_url = 'https://boardgamegeek.com/xmlapi2/thing?id='+BGGID
    response = requests.get(request_url)
    image_url=""
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        image_url = root.find('.//image').text
    else:
        print("Failed to retrieve data from the API")    
    imgDiv="blank"
    imgDiv = soup.find_all(class_ = 'collection_thumbnail')[rank]
    return image_url

    

In [3]:
def getBGGID(soup,rank=1):
    idDiv = soup.find_all(class_ = 'collection_thumbnail')[rank].find('a')['href']
    BGGID = idDiv.split('/')[2]
    return BGGID
    

In [4]:
def Main(interval=1,numResults=1):
    global results_df 
    interval = float(input("Enter the interval (between 0 and 1): "))
    if interval <= 0 or interval > 1:
        interval = float(input("Enter the interval (between 0 and 1): "))

    numResults = int(input("Enter the number of results(max 10): "))
    if numResults >10:
        numResults = int(input("Enter the number of results(max 10): "))   

    weight=1.0
    
    while weight <=5:
        print("finding games for weight "+str(weight)+" ...")
        soup=getSoup(weight)
        for i in range(numResults):
            title = getTitle(soup,i+1)
            image_url=getImageURL(soup,i+1)
            results_df = pd.concat([results_df, pd.DataFrame({"Game Title": [title],"Weight":[weight], "Image URL": [image_url]})], ignore_index=True)
        weight +=interval
             




In [5]:
def getURL(weight):
    weightString = str(weight)
    url = 'https://boardgamegeek.com/search/boardgame?sort=bggrating&sortdir=desc&advsearch=1&floatrange%5Bavgweight%5D%5Bmin%5D='+weightString+'&nosubtypes%5B%5D=boardgameexpansion&floatrange%5Bavgweight%5D%5Bmax%5D='+weightString+'&colfiltertype=&playerrangetype=normal&B1=Submit'
    return url
    

In [6]:
def getSoup(weight):
    url=getURL(weight)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    return soup

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from IPython.display import display, HTML, Image

results_df = pd.DataFrame(columns=["Game Title","Weight", "Image URL"])
Main()
display_df =  results_df[["Game Title"]]
grouped = results_df.groupby("Weight")
organized_df = pd.DataFrame(columns=["Weight", "Title and Image"])

for weight, group in grouped:
    weight_heading = pd.DataFrame({"Weight": f"Weight {weight}", "Title and Image": ""}, index=[0])    
    game_row = pd.DataFrame(columns=["GRW", "GRTI"])
    
    for index, row in group.iterrows():
        game_data = pd.DataFrame({"GRTI": row["Game Title"] + "<br><img src='" + row["Image URL"] + "' height='100px'>"}, index=[0])
        game_row = pd.concat([game_row, game_data], axis=1)
    
    styled_weight = f"<div style='font-size: 32px; color: white; -webkit-text-stroke: 1px black;'>Weight = {weight}</div>"
    display(HTML(styled_weight))
    game_row = game_row.iloc[:, 2:]
    game_row.columns = ["" for _ in range(len(game_row.columns))]
    display(HTML(game_row.to_html(escape=False)))





finding games for weight 1.0 ...


C:\Users\lock4\AppData\Local\Temp\ipykernel_3428\64735567.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({"Game Title": [title],"Weight":[weight], "Image URL": [image_url]})], ignore_index=True)


finding games for weight 1.25 ...
finding games for weight 1.5 ...
finding games for weight 1.75 ...
finding games for weight 2.0 ...
finding games for weight 2.25 ...
finding games for weight 2.5 ...
finding games for weight 2.75 ...
finding games for weight 3.0 ...
finding games for weight 3.25 ...
finding games for weight 3.5 ...
finding games for weight 3.75 ...
finding games for weight 4.0 ...
finding games for weight 4.25 ...
finding games for weight 4.5 ...
finding games for weight 4.75 ...
finding games for weight 5.0 ...


,,,
0,HITSTER,Telestrations After Dark,Catch the Moon


,,,
0,Memoarrr!,The Game: Face to Face,World Championship Russian Roulette


...

,,,
0,Clinic: Deluxe Edition,Great Western Trail: New Zealand,Successors (Fourth Edition)


,,,
0,18CZ,Korea: The Forgotten War,Middle-earth Strategy Battle Game: The Lord of the Rings – Battle of Pelennor Fields


,,,
0,Birds of Prey: Air Combat in the Jet Age,Hurtgen: Hell's Forest,Mistfall: Kickstarter Edition


,,,
0,La Grande Guerre 14-18,Warrior: Miniature Wargame Rules for Ancient and Medieval Warfare 3000BC to 1485AD,Interstellar


,,,
0,Warhammer: The Game of Fantasy Battles (7th Edition),Deluxe Advanced Squad Leader,Bells of War: Conquest Series
